##**Load libraries**

In [ ]:
import pandas as pd
import re

##**Load data from google sheets**

In [ ]:
# # 1st way
# customer_lifespan = pd.read_excel('https://docs.google.com/spreadsheets/d/1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI/export?format=xlsx')
# daily_spendings = pd.read_excel('https://docs.google.com/spreadsheets/d/1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c/export?format=xlsx')
# monthly_expenses = pd.read_excel('https://docs.google.com/spreadsheets/d/10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw/export?format=xlsx')
# payroll = pd.read_excel('https://docs.google.com/spreadsheets/d/1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4/export?format=xlsx')
# receipts_history= pd.read_excel('https://docs.google.com/spreadsheets/d/1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE/export?format=xlsx')

In [ ]:
def read_file_ggsheet(file_id, month_col=None):
  df_ggsheet = pd.read_excel('https://docs.google.com/spreadsheets/d/'+ file_id + '/export?format=xlsx')
  # If the data has datetime columns, we need to filter it and only keep the data for March 2023
  if month_col is not None:
    df_202303 = df_ggsheet[(df_ggsheet[month_col].dt.month == 3) & (df_ggsheet[month_col].dt.year == 2023)]
    return df_202303
  else:
    return df_ggsheet

In [ ]:
customer_lifespan = read_file_ggsheet(file_id = '1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI')
daily_spendings = read_file_ggsheet(file_id='1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c', month_col='date')
monthly_expenses = read_file_ggsheet(file_id='10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw', month_col='month')
payroll = read_file_ggsheet(file_id='1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4', month_col='month')
receipts_history = read_file_ggsheet(file_id='1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE', month_col='date')

In [ ]:
# def read_file_ggsheet(file_id):
#   df_ggsheet = pd.read_excel('https://docs.google.com/spreadsheets/d/'+ file_id + '/export?format=xlsx')
#   return df_ggsheet

In [ ]:
customer_lifespan.describe()

,Unnamed: 0,start_date,churn_date
count,100.000000,100,100
mean,1049.500000,2021-10-09 05:31:12,2022-07-31 11:16:48
min,1000.000000,2020-01-12 00:00:00,2020-08-09 00:00:00
25%,1024.750000,2020-11-12 06:00:00,2022-03-30 00:00:00
50%,1049.500000,2021-11-21 12:00:00,2022-09-19 12:00:00
75%,1074.250000,2022-07-25 18:00:00,2023-02-18 00:00:00
max,1099.000000,2023-03-22 00:00:00,2023-03-31 00:00:00
std,29.011492,NaN,NaN


##**CAC**

- Online advertising:  daily_marketing_spendings -> sum spending
- Sales staff salaries and commissions: payroll -> department in sale, marketing
- Marketing software: monthly expenses -> item = salesforce
- Content creation: đã tính trong chi phí online marketing và payroll.

In [ ]:
# Calculate CAC
# Total sales and marketing expenses
online_ads_cost = daily_spendings['spending'].sum()
sales_salary_cost = payroll[payroll['department'].isin(['Sales','Marketing'])]['paid'].sum()
mkt_software_cost = monthly_expenses[monthly_expenses['item'] == 'Salesforce']['amount'].sum()
total_sale_mkt_expenses = online_ads_cost + sales_salary_cost + mkt_software_cost

In [ ]:
# Number of new customers acquired
new_cust_count = len(receipts_history[receipts_history['new_customer']==1]['customer_id'])
new_cust_count

63

In [ ]:
cac = total_sale_mkt_expenses / new_cust_count
cac

1213.968253968254

###**ARPU**


In [ ]:
#ARPU
revenue_total = receipts_history['receipt_amount'].sum()
num_cust = len(receipts_history['customer_id'].unique())


In [ ]:
arpu = (revenue_total)/(num_cust)
arpu

284.3595890410959

**COGS**
- Server (and other infrastructure costs) + Software licenses: monthly expenses -> item in (AWS, Google Cloud Storage, Atlassian Jira, Slack, Zoom
- Salary of the employees that directly involved into product development: payroll -> department = Engineering.

In [ ]:
payroll.head()

,month,department,employee_name,position,paid
34,2023-03-01,Sales,John Doe,Sales Manager,1500
35,2023-03-01,Sales,Jane Smith,Sales Associate,600
36,2023-03-01,Sales,Jim Brown,Sales Associate,700
37,2023-03-01,Sales,Laura Miller,Sales Associate,800
38,2023-03-01,Marketing,Alice Johnson,Marketing Manager,1650


In [ ]:
#COGS
infrastructure_costs = monthly_expenses[monthly_expenses['item'].isin(['AWS Hosting','Google Cloud Storage','Atlassian Jira','Slack','Zoom'])]['amount'].sum()
salary_employees_involved = payroll[payroll['department'] == 'Engineering']['paid'].sum()

In [ ]:
cogs = infrastructure_costs + salary_employees_involved
cogs

20840

**Gross Margin**

Gross Margin = ((Revenue - COGS)/Revenue *100

In [ ]:
  gross_margin = ((revenue_total-cogs)/revenue_total)*100

In [ ]:
gross_margin

74.9015451687883

**LTV**

LTV = ARPU * CustomerLifeSpan * Gross Margin

In [ ]:
customer_lifespan['cust_lifespan(months)'] = (((customer_lifespan['churn_date'] - customer_lifespan['start_date']).dt.days )/30).astype('int')


In [ ]:
customer_lifespan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             100 non-null    int64         
 1   start_date             100 non-null    datetime64[ns]
 2   churn_date             100 non-null    datetime64[ns]
 3   cust_lifespan(months)  100 non-null    int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 3.3 KB


In [ ]:
a = customer_lifespan['cust_lifespan(months)'].mean()
a

9.32

In [ ]:
LTV = arpu * customer_lifespan['cust_lifespan(months)'].mean() * (gross_margin/100)

In [ ]:
LTV

1985.0642465753424

**LTV/CAC**


In [ ]:
  ltv_cac = LTV / cac

In [ ]:
ltv_cac

1.6351862909812576

##**Conclusion**##


**Insights**
- LTV/CAC = **1.6352** (Not optimal). Although this metric is greater than 1, but not ideal (usually >= **3**). This shows that customers give more revenue than the cost of acquiring them, but the gross margin is not really optimal.

- High CAC (**$1213.97**). The cost of acquiring a customer is quite high, reducing the profit per customer.

- Gross margin = **74.90%** (Good). The gross profit margin is high, indicating that the business model is profitable, but the total operating costs need to be considered to ensure that the final profit remains sustainable.

- ARPU = **$284.36** (Low compared to CAC). The average revenue per customer is not high enough compared to the cost of acquiring customers (CAC). This can affect the ability to recover capital quickly.

- High COGs (**$20,840**). Cost of goods sold is very high, which can affect overall profitability.

**Actionable insights**
- Increase LTV: Retain customers longer, encourage ugrades to higher packages, improve service.
- Reduce CAC: Leverage content marketing, optimize advertising campaigns, promote referrals.
- Increase ARPU: Sell more add-ons, expand premium packages, more flexible pricing.
- Optimize COGs: Cut cloud costs, automate backend, optimize engineer performance.
- Improve Gross Margin: Increase revenue from B2B customers, improve self-service support.